#### COPY ALL DCM IMAGES INTO FROM stage_2_test_images INTO stage_2_train_images

In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [2]:
root_path = '/home/phoenix/Desktop/ITU/Trustable AI/Project/pneumonia-detection-pytorch-master/'

In [3]:
def get_bb(img_t):
    for rec in train_labels_df[train_labels_df['patientId'] == img_t].values.tolist():
        try:
            return (rec[1], rec[2], rec[3], rec[4])
        except:
            return (None)

In [4]:
balanced_label_df = pd.read_csv(root_path + 'dataset/balanced_label.txt')
balanced_label_df

,/home/phoenix/Desktop/ITU/Trustable AI/Project/pneumonia-detection-pytorch-master/dataset/rsna-pneumonia-detection-challenge/stage_2_train_images/ac69ac97-4835-40b9-a4a7-442626dd28e9.dcm,2
0,/home/phoenix/Desktop/ITU/Trustable AI/Project...,2
1,/home/phoenix/Desktop/ITU/Trustable AI/Project...,2
2,/home/phoenix/Desktop/ITU/Trustable AI/Project...,2
3,/home/phoenix/Desktop/ITU/Trustable AI/Project...,2
4,/home/phoenix/Desktop/ITU/Trustable AI/Project...,2
...,...,...
26394,/home/phoenix/Desktop/ITU/Trustable AI/Project...,1
26395,/home/phoenix/Desktop/ITU/Trustable AI/Project...,1
26396,/home/phoenix/Desktop/ITU/Trustable AI/Project...,1
26397,/home/phoenix/Desktop/ITU/Trustable AI/Project...,1


In [5]:
TRAIN_SAMPLES = 26400
detailed_class_info_df = pd.read_csv(root_path + 'dataset/stage_2_detailed_class_info.csv')
detailed_class_info_df

,patientId,class
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,No Lung Opacity / Not Normal
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,No Lung Opacity / Not Normal
2,00322d4d-1c29-4943-afc9-b6754be640eb,No Lung Opacity / Not Normal
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,Normal
4,00436515-870c-4b36-a041-de91049b9ab4,Lung Opacity
...,...,...
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,Lung Opacity
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,Normal
30224,c1f6b555-2eb1-4231-98f6-50a963976431,Normal
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,Lung Opacity


In [6]:
train_labels_df = pd.read_csv(root_path + 'dataset/stage_2_train_labels.csv')
train_labels_df

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1
...,...,...,...,...,...,...
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,185.0,298.0,228.0,379.0,1
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,NaN,NaN,NaN,NaN,0
30224,c1f6b555-2eb1-4231-98f6-50a963976431,NaN,NaN,NaN,NaN,0
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,570.0,393.0,261.0,345.0,1


In [7]:
detailed_class_info_df = detailed_class_info_df.groupby('class').apply(lambda x: x.sample(TRAIN_SAMPLES//3).reset_index(drop=True))
detailed_class_info_df

patientId         class
class                                                                
Lung Opacity 0     50d934dd-f974-48c2-b021-f40b9539051e  Lung Opacity
             1     86469e34-c9ba-4ec9-bbe0-6d04535122a0  Lung Opacity
             2     3d505bb2-544e-4e2d-8859-ef33bf31c49f  Lung Opacity
             3     4ee60076-67b2-46f8-965e-29c167a30cd3  Lung Opacity
             4     8759fcb4-1a14-4724-959d-235060495181  Lung Opacity
...                                                 ...           ...
Normal       8795  bf597ebf-1de4-4b04-bf23-6f036c4b1440        Normal
             8796  928fedb8-0cf6-447f-b6f3-292ac06b2b35        Normal
             8797  4e53d257-6eae-4ace-951f-72115498fbe7        Normal
             8798  cb398b60-eb34-40f4-aac9-5e179d5c02e4        Normal
             8799  6a2e71ca-55a4-4b9e-ae97-a47a070fc89d        Normal

[26400 rows x 2 columns]

In [8]:
#np.savetxt(r'/home/tianshu/pneumonia/dataset/balanced_label.txt', df.values, fmt='%s')
f = open(root_path + 'dataset/balanced_label.txt', 'w')
arr = detailed_class_info_df.values
new_line = ''
for index in tqdm(range(arr.shape[0])):
    line = arr[index]
    img = line[0]
    label = line[1]
    if label=='No Lung Opacity / Not Normal':
        label = '0'
    elif label=='Normal':
        label = '1'
    elif label=='Lung Opacity':
        label= '2'
    
    new_line = root_path + 'dataset/rsna-pneumonia-detection-challenge/stage_2_train_images/' + img + '.dcm' + ',' + label + '\n'
    f.write(new_line)

f.close()


100%|██████████| 26400/26400 [00:00<00:00, 918064.75it/s]


In [9]:
info_0 = []
info_1 = []
info_2 = []

with open(root_path + 'dataset/balanced_label.txt') as file:
    for line in file:
        cur_line = line.rstrip().split(",")
        if cur_line[1] == '0':         
            info_0.append(cur_line)
        elif cur_line[1] == '1':
            info_1.append(cur_line)
        elif cur_line[1] == '2':
            info_2.append(cur_line)


In [10]:
#info_0_train, info_0_test, _, _ = train_test_split(info_0, info_0, test_size=0.33, random_state=42)
#len(info_0_train), len(info_0_test)

In [11]:
#info_1_train, info_1_test, _, _ = train_test_split(info_1, info_1, test_size=0.33, random_state=42)
#len(info_1_train), len(info_1_test)

In [12]:
#info_2_train, info_2_test, _, _ = train_test_split(info_2, info_2, test_size=0.33, random_state=42)
#len(info_2_train), len(info_2_test)

In [13]:
def splitData(data_list0, data_list1, data_list2, n_splits=3):
    def getChunk(data_list, indexes):
        return_list = []
        for i in indexes:
            return_list.append(data_list[i])
        return return_list
    
    def writeFile(data, data_name):
        textfile = open(data_name, "w")
        for element in data:
            #print(element)
            textfile.write(str(element) + "\n")
        textfile.close()        
        
    kf = KFold(n_splits=n_splits)
    for fold_no, (train_index, test_index) in enumerate(kf.split(data_list0)): # doesn't matter if;
        # it is data_list0 or data_list1 or data_list0. because we use it only for getting indexes.
        #print("FOLD NO: ", fold_no, "TRAIN:", train_index, "TEST:", test_index)
        X_train_0 = getChunk(data_list0, train_index)
        X_test_0 = getChunk(data_list0, test_index)
        X_train_1 = getChunk(data_list1, train_index)
        X_test_1 = getChunk(data_list1, test_index)
        X_train_2 = getChunk(data_list2, train_index)
        X_test_2 = getChunk(data_list2, test_index)
        
        X_train = X_train_0 + X_train_1 + X_train_2
        X_test = X_test_0 + X_test_1 + X_test_2
        
        writeFile(X_train, "dataset/X_train_fold_" + str(fold_no) + ".txt")
        writeFile(X_test, "dataset/X_test_fold_" + str(fold_no) + ".txt")
            

In [14]:
splitData(info_0, info_1, info_2)